
# FINN Preprocessor: Local application, Preloaded MODIS raster

## 1. Setting Envoronments

### Systems settings

Most likely no need to be edited.

In [ ]:
# python libraries
import sys
import os
import re
import glob
import datetime
import subprocess
import shlex
from urllib.parse import urlparse
from importlib import reload
import gdal
import matplotlib.pylab as plt


# finn preproc codes
sys.path = sys.path + ['../code_anaconda']
import downloader
import af_import
import rst_import
import polygon_import
import run_step1
import run_step2
import export_shp
import plotter

In [ ]:
# database settings
os.environ['PGDATABASE'] = 'gis'
os.environ['PGUSER'] = 'finn'
os.environ['PGPASSWORD'] = 'finn'
os.environ['PGHOST'] = 'localhost'
os.environ['PGPORT'] = '5432'

Make sure that the PostGIS database is ready.

In [ ]:
# show info for the database
!psql postgres -c 'SELECT version();'
!pg_lsclusters

In [ ]:
# TODO i want to move this to Dockerfile somehow
# create plpython, needed only once for the database
try:
    p = subprocess.run(shlex.split("psql -d %s -c 'CREATE LANGUAGE plpython3u;'" % os.environ['PGDATABASE']), 
                       check=True, stderr=subprocess.PIPE)
except subprocess.CalledProcessError as e:
    if 'already exists' in e.stderr.decode():
        print(e.stderr.decode().replace('ERROR','OK').strip())
        pass

### Settings for ActiveFire Dataset

You specify two items related active filre.
1. `tag_af`: a string that identifies this active fire dataset through the processing.
2. `af_fnames`: list of path for the shape files you downloaded from FIRMS website  
Files you specified as `af_names` will be imported into a databse schema "af_<i>tag_af</i>", and processed in the database.  
Final output file will be "out_<i>tag_af</i>_*.csv".
3. `firstday`, `lastday`: (optional) you specify first/last year of analysis

In [ ]:
# tag to identify active fire dataset
# USA (excl. ALK) 7 days fire downloaded 2019-01-13 
tag_af = 'testUSA_012019'

# shp file names
af_fnames = [
    '../downloads/firms/testUSA_012019/MODIS_C6_USA_contiguous_and_Hawaii_7d.shp',
    '../downloads/firms/testUSA_012019/VNP14IMGTDL_NRT_USA_contiguous_and_Hawaii_7d.shp',
]

# You can either set fist and last day of analysis by your self here,
# firstday = datetime.date(year, 1, 1)
# lastday = datetime.date(year+1, 1, 1)

# or use the period covered by the input active fire file (keep both variables to None)
firstday = None
lastday = None

This particular sample AF dataset are provided by the developper.  In other applications, it will be user's resoponsibility to provide shape file for active fire in specified path/name.

In [ ]:
# check input file exists
if any([not os.path.exists(_) for _ in af_fnames]):
    print("Not found:\n" + '\n'.join(af_fnames) + '\n\n')
    raise RuntimeError('AF inputs not found')

### Settings for Land Surface Datasets (land cover, vegetation continuous field, region definieons)

MODIS land cover types, MODIS vegetation continuous field will be downloaded, if needed, for the region of AF input
1. `year_rst`: MODIS raster data year to be used for the analysis

Other parameters such as `tag_lct`, `tag_vcf`, `tag_regnum` (identify landcover, vcf and region number dataset) are set automatically for MODIS dataset.  
The datasets are imported into database schema "raster", with table names "rst_<i>tag_lct</i>", or "rst_modlct_2017", for example.  
An overview raster "o_32_rst_modlct_2007" is created as well, as the real dataset is difficult to handle for QA.

In [ ]:
# MODIS raster datasets' year
year_rst = 2017
# or you can match it to years of active fire file
#year_rst = None

In [ ]:
if year_rst is None:
    if any(_ is None for _ in (firstday, lastday)):
        # read from shape file
        raise RuntimeError("year_rst speficication from AF data has not implemented yet")
    else:
        # assume first day of analysis is year of interest
        year_rst = firstday.year

In [ ]:
# tag to identify datasets, automatically set to be modlct_YYYY, modvcf_YYYY
tag_lct = 'modlct_%d' % year_rst
tag_vcf = 'modvcf_%d' % year_rst

# tag for the region number polygon
tag_regnum = 'regnum'

# definition of variables in the raster files
rasters = [
        {
            'tag': tag_lct,
            'kind': 'thematic',
            'variable': 'lct'
        },
        {
            'tag': tag_vcf,
            'kind': 'continuous',
            'variables': ['tree', 'herb', 'bare'],
        },
        {
            'tag': tag_regnum,
            'kind': 'polygons',
            'variable_in': 'region_num',
            'variable': 'regnum',
        },
]

## 2. Import active fire data

Go ahead and import into database.

<b>Be careful!!</b> The code has no safe guard and wipe the schema for the scheama "af_<i>tag_af</i>" and starts over.  

Let me think the design a bit more for now.

In [ ]:
reload(af_import)

# TODO this is destructive need to safe guard!
# tell user schema is there, list table names and # of row of each.  Ask her to delete manually or something to proceed
af_import.main(tag_af, af_fnames)

print()
for i,fn in enumerate(af_fnames):
    print(fn)
    p = subprocess.run(['psql', '-c', 'select count(*) from "af_%s".af_in_%d;' % (tag_af, i+1)], stdout=subprocess.PIPE)
    print(p.stdout.decode())


In [ ]:
# confirm that raster data covers extent of AF data
reload(af_import)
dct = {}
for i,fn in enumerate(af_fnames):
    for tag_rst in (tag_lct, tag_vcf):
        cnts = af_import.check_raster_contains_fire(
            '"raster"."skel_rst_%s"' % tag_rst, 
            '"af_%s"."af_in_%d"' % (tag_af, i+1)
        )
        print(os.path.basename(fn), tag_rst, cnts)
        dct[(fn,tag_rst)] = cnts

if any(_['n_not_contained'] > 0 for _ in  dct.values()):
    print('Some fire is not conained in raster')
    raise RuntimeError('Some fire is not conained in raster.  Reload raster!!')
    pass


## 3. Process active fire data

### Running "step 1" grouping points

In [ ]:
reload(run_step1)
run_step1.main(tag_af, firstday=firstday, lastday=lastday, ver='v7m', run_prep = True, run_work=True)

### Running "step 2" intersection with raster datasets

In [ ]:
reload(run_step2)

assert run_step2.ver == 'v8b'
run_step2.main(tag_af, rasters, firstday=firstday, lastday=lastday)

## 5. Export the output

Default output directory is this diretory (where you have this Jupyter Notebook file), and output file has long name of having tag of each datasets.

In [ ]:
outdir = '.'
shpname = 'out_{0}_{1}_{2}_{3}.shp'.format(tag_af, tag_lct, tag_vcf, tag_regnum)

In [ ]:
schema = 'af_' + tag_af
tblname = 'out_{0}_{1}_{2}'.format(tag_lct, tag_vcf, tag_regnum)
flds = ('v_lct', 'f_lct', 'v_tree', 'v_herb', 'v_bare', 'v_regnum')

In [ ]:
reload(export_shp)
export_shp.main(outdir, schema, tblname, flds, shpname)